<a href="https://colab.research.google.com/github/PhamBaVuong/PhamBaVuong_DATN/blob/main/Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install openrouteservice

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import openrouteservice as ors
ors_key = '5b3ce3597851110001cf6248e0d3f77215c44c9fa97ee9d696e36133'
client = ors.Client(key = ors_key)

In [21]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

coord = [[10.841712258705531,106.66482746593148], [10.827695475879587,106.70004872412872]]

route = client.directions(coord, profile='driving-car', format='geojson')

#Define coordinates of where we want to center our map
boulder_coords = [10.841712258705531, 106.66482746593148]

#Create the map
my_map = folium.Map(location = boulder_coords, zoom_start = 5)

folium.GeoJson(route, name='route').add_to(my_map)

folium.LayerControl().add_to(my_map)

#Display the map
my_map

ApiError: ignored

In [ ]:
pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 4.5 MB/s 
     |████████████████████████████████| 1.0 MB 40.6 MB/s 
     |████████████████████████████████| 11.2 MB 53.2 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 6.3 MB 41.8 MB/s 
     |████████████████████████████████| 1.0 MB 31.8 MB/s 
     |████████████████████████████████| 16.7 MB 56.6 MB/s 
     |████████████████████████████████| 959 kB 72.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [ ]:
import osmnx as ox
import networkx as nx

ox.config(log_console=True, use_cache=True)

G_walk = ox.graph_from_place('Manhattan Island, New York City, New York, USA',
                             network_type='walk')

orig_node = ox.get_nearest_node(G_walk,
                                (40.748441, -73.985664))

dest_node = ox.get_nearest_node(G_walk,
                                (40.748441, -73.4))

route = nx.shortest_path(G_walk,
                         orig_node,
                         dest_node,
                         weight='length')

route_map = ox.plot_route_folium(G_walk, route)

/usr/local/lib/python3.7/dist-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


In [ ]:
from flask import Flask
import folium

app = Flask(__name__)


@app.route("/")
def base():
    map = folium.Map(
        location = [10.841712258705531, 106.66482746593148]
    )
    return map._repr_html_()

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [ ]:
import folium
from folium import plugins

coord = [[10.8028976, 106.6752956], [10.827695475879587, 106.70004872412872]]


#Define coordinates of where we want to center our map
boulder_coords = [10.841712258705531, 106.66482746593148]

#Create the map
my_map = folium.Map(location = boulder_coords, zoom_start = 5)
# added lat long to route
route_lats_longs = [[10.841712258705531, 106.66482746593148],    # Jamnagar
                    [10.827695475879587, 106.70004872412872]]    # Jaipur
# Ploting ant-route
plugins.AntPath(route_lats_longs).add_to(my_map)

my_map

In [7]:
import requests

url = "https://route-and-directions.p.rapidapi.com/v1/routing"

querystring = {"waypoints":"48.34364,10.87474|48.37073,10.90925","mode":"drive"}

headers = {
	"X-RapidAPI-Key": "e01a31e380mshc4c39b600d13ae1p10ce1djsn779af9ef8efb",
	"X-RapidAPI-Host": "route-and-directions.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)


{"message":"You are not subscribed to this API."}


In [ ]:
import requests
def get_directions_response(lat1, long1, lat2, long2, mode='drive'):
   url = "https://route-and-directions.p.rapidapi.com/v1/routing"
   key = "e01a31e380mshc4c39b600d13ae1p10ce1djsn779af9ef8efb"
   host = "route-and-directions.p.rapidapi.com"
   headers = {"X-RapidAPI-Key": key, "X-RapidAPI-Host": host}
   querystring = {"waypoints":f"{str(lat1)},{str(long1)}|{str(lat2)},{str(long2)}","mode":mode}
   response = requests.request("GET", url, headers=headers, params=querystring)
   return response
response = get_directions_response(52.4013, 4.5425, 52.402, 4.5426)

In [ ]:
import folium
def create_map(response):
   # use the response
   mls = response.json()['geometry']['coordinates']
   points = [(i[1], i[0]) for i in mls[0]]
   m = folium.Map()
   # add marker for the start and ending points
   for point in [points[0], points[-1]]:
      folium.Marker(point).add_to(m)
   # add the lines
   folium.PolyLine(points, weight=5, opacity=1).add_to(m)
   # create optimal zoom
   df = pd.DataFrame(mls[0]).rename(columns={0:'Lon', 1:'Lat'})[['Lat', 'Lon']]
   sw = df[['Lat', 'Lon']].min().values.tolist()
   ne = df[['Lat', 'Lon']].max().values.tolist()
   m.fit_bounds([sw, ne])
   return m
m = create_map(response)

In [15]:
pip install polyline geocoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 2.1 MB/s 


In [16]:
import logging
import requests
import json
import polyline
import folium
from folium.plugins import MeasureControl
import geocoder

from functools import lru_cache

logger = logging.getLogger(__name__)
DEBUG = True

@lru_cache(maxsize=None)
def geocode(location):
    return _geocode(location)

def _geocode(location):
    import geocoder
    g = geocoder.osm(location)
    return g.latlng

@lru_cache(maxsize=None)
def get_route(olat, olng, dlat, dlng):
    response = _get_route(olat, olng, dlat, dlng)
    return response

def _get_route(olat, olng, dlat, dlng):
    url = f'http://router.project-osrm.org/route/v1/driving/{olng},{olat};{dlng},{dlat}?alternatives=false&steps=false'
    # logger.debug(url)
    response = None

    try:
        logger.debug(f'====== OSRM: {url}')
        response = requests.get(url, verify=False)
    except Exception as ex:
        raise

    # logger.debug(response.text)
    if response and response.text:
        response_dict = json.loads(response.text)
        #possible = pd.DataFrame([{'Distance': (route['distance'] / 1000) *  0.621371 , route['weight_name']: route['weight']} for route in response_dict['routes']])
        return response_dict
    else:
        return None

def get_routing_map(origin, destination, zoom=5):
    orig_latlng = geocode(origin)
    dest_latlng = geocode(destination)

    resp = get_route(orig_latlng[0], orig_latlng[1], dest_latlng[0], dest_latlng[1])

    decoded = polyline.decode(resp["routes"][0]['geometry'])
    distance = resp["routes"][0]['distance'] * 0.000621371
    duration = resp["routes"][0]['duration'] / 60

    map2 = folium.Map(location=(orig_latlng[0], orig_latlng[1]), zoom_start=zoom,
                                    control_scale=True)
    # map2.add_child(MeasureControl(
    #     primary_length_unit='miles',
    #     secondary_length_unit='meters',
    #     primary_area_unit='acres',
    #     secondary_area_unit='sqmeters')
    # )

    folium.PolyLine(locations=decoded, color="blue").add_to(map2)

    print(f"{duration} minutes")
    print(f"{distance} miles")

    return map2

In [22]:
map = get_routing_map('Plymouth, MN', 'San Diego, CA', zoom=5)
map

2061.8616666666667 minutes
1966.7043346808 miles
